# README
## Options:
### Pretrained model:
- word2vec-ruscorpora-300	
- glove-twitter-200

### Tokenization: How to manage OOV?
- Link: '&'
- Tag: '#'
- Mention: '@' 

In [229]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', 400)

## Collective Code

In [11]:
import copy
import pandas as pd
import numpy as np
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from nltk.tokenize import TweetTokenizer


import gensim
import gensim.models.word2vec as w2v
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [13]:
import gensim.downloader as api
model = api.load('glove-twitter-200')

In [20]:
w2v_object = model.wv
w2v_vectors = w2v_object.vectors # here you load vectors for each word in your model
w2v_indices = {word: w2v_object.vocab[word].index for word in w2v_object.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

<ipython-input-20-77925cc2d5b2>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  w2v_object = model.wv


In [21]:
def vectorize(line):
    words = []
    for word in line:  # line - iterable, for example list of tokens
        try:
            w2v_idx = w2v_indices[word]
        except KeyError:  # if you does not have a vector for this word in your w2v model, continue
            words.append(list(np.zeros(200,)))
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)


def get_W2V_AVG(raw_data):
    tweet_tokenizer = TweetTokenizer()
    tweet_tokens = []
    stop_words = set(stopwords.words('english'))

   
    for sent in raw_data.text:
        sent = re.sub(r"http\S+", "&", sent)
        sent = re.sub(r"@\S+", "@", sent)
        sent = re.sub(r"#\S+", "#", sent)
        sent = re.sub(r'([^\s\w@#&]|_)+', '', sent)
        # sent = re.sub('', '', sent.lower())
        # print(tweet_tokenizer.tokenize(sent))
        # sent = [tweet_tokenizer.tokenize(sent)]
        sent = [tweet_tokenizer.tokenize(sent.lower())]
        temp = [token for token in sent[0] if not token in stop_words]
        tweet_tokens.append([temp])
        # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
    df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])
    df_tokens['token_vec'] = copy.deepcopy(df_tokens['token'])

    for index, sent in enumerate(df_tokens['token_vec']):
        df_tokens['token_vec'][index] = vectorize(sent).mean(axis=0)

    df_temp = pd.DataFrame(
        df_tokens['token_vec'].values.tolist()).add_prefix('vec_avg')

    df_tokens = df_tokens.join(df_temp).drop('token_vec', axis=1)
    return pd.DataFrame(df_tokens)


In [22]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")
raw_RHI = pd.read_csv("./data/_RHI_text.csv")

In [23]:
PHEME_W2V_AVG = get_W2V_AVG(raw_data)
RHI_W2V_avg = get_W2V_AVG(raw_RHI)
# RHI_W2V_avg

In [28]:
# Checking the result
# PHEME_W2V_AVG.head(3)

In [30]:
PHEME_W2V_AVG.to_csv('./data/_PHEME_text_AVGw2v.csv', index = False)
RHI_W2V_avg.to_csv('./data/_RHI_text_AVGw2v.csv', index = False)

In [326]:
raw_RHI = pd.read_csv("./data/_RHS_text_AVGw2v.csv")

In [327]:
raw_RHI.head()

,token,vec_avg0,vec_avg1,vec_avg2,vec_avg3,vec_avg4,vec_avg5,vec_avg6,vec_avg7,vec_avg8,vec_avg9,vec_avg10,vec_avg11,vec_avg12,vec_avg13,vec_avg14,vec_avg15,vec_avg16,vec_avg17,vec_avg18,vec_avg19,vec_avg20,vec_avg21,vec_avg22,vec_avg23,vec_avg24,vec_avg25,vec_avg26,vec_avg27,vec_avg28,vec_avg29,vec_avg30,vec_avg31,vec_avg32,vec_avg33,vec_avg34,vec_avg35,vec_avg36,vec_avg37,vec_avg38,vec_avg39,vec_avg40,vec_avg41,vec_avg42,vec_avg43,vec_avg44,vec_avg45,vec_avg46,vec_avg47,vec_avg48,vec_avg49,vec_avg50,vec_avg51,vec_avg52,vec_avg53,vec_avg54,vec_avg55,vec_avg56,vec_avg57,vec_avg58,vec_avg59,vec_avg60,vec_avg61,vec_avg62,vec_avg63,vec_avg64,vec_avg65,vec_avg66,vec_avg67,vec_avg68,vec_avg69,vec_avg70,vec_avg71,vec_avg72,vec_avg73,vec_avg74,vec_avg75,vec_avg76,vec_avg77,vec_avg78,vec_avg79,vec_avg80,vec_avg81,vec_avg82,vec_avg83,vec_avg84,vec_avg85,vec_avg86,vec_avg87,vec_avg88,vec_avg89,vec_avg90,vec_avg91,vec_avg92,vec_avg93,vec_avg94,vec_avg95,vec_avg96,vec_avg97,vec_avg98,vec_avg99,vec_avg100,vec_avg101,vec_avg102,vec_avg103,vec_avg104,vec_avg105,vec_avg106,vec_avg107,vec_avg108,vec_avg109,vec_avg110,vec_avg111,vec_avg112,vec_avg113,vec_avg114,vec_avg115,vec_avg116,vec_avg117,vec_avg118,vec_avg119,vec_avg120,vec_avg121,vec_avg122,vec_avg123,vec_avg124,vec_avg125,vec_avg126,vec_avg127,vec_avg128,vec_avg129,vec_avg130,vec_avg131,vec_avg132,vec_avg133,vec_avg134,vec_avg135,vec_avg136,vec_avg137,vec_avg138,vec_avg139,vec_avg140,vec_avg141,vec_avg142,vec_avg143,vec_avg144,vec_avg145,vec_avg146,vec_avg147,vec_avg148,vec_avg149,vec_avg150,vec_avg151,vec_avg152,vec_avg153,vec_avg154,vec_avg155,vec_avg156,vec_avg157,vec_avg158,vec_avg159,vec_avg160,vec_avg161,vec_avg162,vec_avg163,vec_avg164,vec_avg165,vec_avg166,vec_avg167,vec_avg168,vec_avg169,vec_avg170,vec_avg171,vec_avg172,vec_avg173,vec_avg174,vec_avg175,vec_avg176,vec_avg177,vec_avg178,vec_avg179,vec_avg180,vec_avg181,vec_avg182,vec_avg183,vec_avg184,vec_avg185,vec_avg186,vec_avg187,vec_avg188,vec_avg189,vec_avg190,vec_avg191,vec_avg192,vec_avg193,vec_avg194,vec_avg195,vec_avg196,vec_avg197,vec_avg198,vec_avg199
0,"['breaking', 'armed', 'man', 'takes', 'hostage', 'kosher', 'grocery', 'east', 'paris', '&']",-0.245181,-0.01027,0.116429,0.159705,-0.100017,-0.088065,0.242909,-0.043744,0.086057,-0.133616,-0.165871,0.124254,-0.656505,0.249567,0.171354,-0.020197,0.048338,-0.104669,0.016054,-0.262492,0.097819,-0.144589,-0.118979,-0.075988,0.009608,0.774872,0.189386,-0.051733,-0.148943,-0.029019,-0.146035,-0.113401,0.016945,-0.157495,0.159048,0.358597,-0.050946,0.079987,0.305156,-0.110494,0.123272,0.001231,0.016253,-0.106919,0.030864,0.152466,0.074397,-0.033293,-0.058297,0.053371,-0.50394,0.008548,-0.090691,0.069851,-0.23457,0.151345,-0.198295,0.01931,-0.06298,0.08632,0.111909,0.095357,0.076761,-0.008932,0.170908,-0.052601,0.182794,0.083385,-0.042883,-0.031243,-0.117652,-0.030746,0.118263,0.065533,0.132332,0.052125,-0.008038,-0.022878,-0.028015,-0.04078,0.381744,-0.078668,0.191174,0.14731,-0.098038,-0.14875,-0.117466,0.215932,-0.029577,-0.114086,-0.015651,-0.02498,0.155709,0.089995,-0.089634,0.115466,0.040305,-0.009728,0.013069,0.034053,0.282391,-0.129498,-0.119499,-0.273537,0.178535,-0.301966,0.080896,0.071021,0.123112,-0.173997,0.074667,-0.099763,0.070493,0.01303,0.003937,-0.235772,-0.051655,0.141754,-0.180339,0.307101,0.001445,-0.023391,-0.060489,0.103714,-0.207719,-0.1599,0.07702,0.156897,0.366401,0.327158,-0.244054,0.089152,-0.127359,-0.480328,-0.016111,-0.284418,0.091102,0.058873,0.133842,-0.068824,-0.053446,0.227917,-0.200994,-0.189614,-0.152296,-0.004471,-0.018664,-0.084851,-0.008542,0.144377,0.03496,-0.080191,-3.096044,-0.072925,0.216484,-0.036533,-0.013923,0.101394,0.125695,-0.034054,-0.160577,0.173289,0.001495,0.011303,0.187776,-0.171213,-0.123493,-0.052816,0.324482,-0.222219,-0.24465,0.051662,-0.038799,0.183936,-0.046773,-0.591883,0.137641,-0.120051,-0.324571,0.191564,-0.13271,-0.163548,-0.253983,0.091246,0.114669,0.025024,0.130568,-0.295191,0.089739,0.198975,0.063809,-0.1811

# Import data

In [219]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")

In [9]:
print(raw_data.shape)
raw_data.sample(5)

(5802, 2)


,text,Event
1080,"#ICYMI, Christopher Hitchens on the case for m...",charliehebdo
4120,DEVELOPING NEWS: Soldier shot at War Memorial....,ottawashooting
5004,RT @tomsteinfort: Terrifying photo of hostages...,sydneysiege
2305,#Ferguson chief said the officer was unaware o...,ferguson
4898,BREAKING: 2 people have run out of Sydney buil...,sydneysiege


# Preprocessing

# Tokenization

In [243]:
tweet_tokenizer = TweetTokenizer()
tweet_tokens = []

for sent in raw_data.text:
    sent = re.sub(r"http\S+", "&", sent)
    sent = re.sub(r"@\S+", "@", sent)
    sent = re.sub(r"#\S+", "#", sent)
    sent = re.sub(r'([^\s\w@#&]|_)+','', sent)
    sent = re.sub('','', sent.lower())
    # print(tweet_tokenizer.tokenize(sent))
    sent = [tweet_tokenizer.tokenize(sent)]
    # sent = [tweet_tokenizer.tokenize(sent.lower())]
    tweet_tokens.append(sent)
    # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])


In [237]:
df_tokens.head()

,token
0,"[breaking, armed, man, takes, hostage, in, kosher, grocery, east, of, paris, &]"
1,"[#, killers, dead, confirmed, by, gendarmerie]"
2,"[top, french, cartoonists, charb, cabu, wolinski, tignous, confirmed, among, dead, in, #, #, attack, editor, is, critically, wounded]"
3,"[police, have, surrounded, the, area, where, the, #, attack, suspects, are, believed, to, be, &, &]"
4,"[photo, armed, gunmen, face, police, officers, near, #, hq, in, paris, &, &]"


# Word2Vec

## Downloading Pretrained model for Gensim

## Fetching pretrained Model and Convert the Raw Text

In [230]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
# corpus = api.load('text8')
# wv = api.load('word2vec-google-news-300')
# fasttext-wiki-news-subwords-300'
#  'glove-twitter-200',
model = api.load('glove-twitter-200')

In [20]:
# import inspect
# print(inspect.getsource(wv.__class__))

In [42]:
w2v_object = model.wv
w2v_vectors = w2v_object.vectors # here you load vectors for each word in your model
w2v_indices = {word: w2v_object.vocab[word].index for word in w2v_object.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [241]:
def vectorize(line): 
    words = []
    for word in line: # line - iterable, for example list of tokens 
        try:
            w2v_idx = w2v_indices[word]
        except KeyError: # if you does not have a vector for this word in your w2v model, continue 
            words.append(list(np.zeros(200,)))
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)

In [233]:
# print("Tweet 1: ", raw_data['text'][1])
# print("Indice of '{}': {}".format(df_tokens['token'][1][0], w2v_indices[df_tokens['token'][1][0]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][0], w2v_vectors[w2v_indices[raw_data['text_token'][1][0]]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_indices[raw_data['text_token'][1][1]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_vectors[w2v_indices[raw_data['text_token'][1][1]]]))
# # print("\nVector of the first headline:\n", vectorize(raw_data['text_token'][1]))

In [244]:
import copy
df_tokens['token_vec'] = copy.deepcopy(df_tokens['token'])

for index, sent in enumerate(df_tokens['token_vec']):
    df_tokens['token_vec'][index] = vectorize(sent).mean(axis=0)

# df_test[['text_token','text_token_vec']].head()

df_temp = pd.DataFrame(df_tokens['token_vec'].values.tolist()).add_prefix('vec_avg')

df_tokens = df_tokens.join(df_temp).drop('token_vec',axis=1)
# df_test.drop('text_token_vec',axis=1, inplace=True)

In [1]:
df_tokens.head()

NameError: name 'df_tokens' is not defined